<a href="https://colab.research.google.com/github/eugenepeley/EMP-ATM-BANK-MACHINE-/blob/master/Robotics2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def compute_attractive_field(X, Y, goal):
    Fx_att = goal[0] - X
    Fy_att = goal[1] - Y
    return Fx_att, Fy_att

def compute_repulsive_field(X, Y, obstacle_pos, kr=10, repulsion_range=2.0, shape='circle'):
    dx = X - obstacle_pos[0]
    dy = Y - obstacle_pos[1]

    if shape == 'ellipse':
        # Non-circular obstacle (simple ellipse): scale axes
        dx = dx / 1.5  # stretch x-axis
        dy = dy / 0.75  # shrink y-axis

    distance = np.sqrt(dx**2 + dy**2)
    with np.errstate(divide='ignore', invalid='ignore'):
        strength = kr * (1.0 / distance - 1.0 / repulsion_range)
        strength[distance > repulsion_range] = 0
        strength = np.clip(strength, 0, None)
        Fx_rep = strength * dx / (distance**2)
        Fy_rep = strength * dy / (distance**2)
        Fx_rep[np.isnan(Fx_rep)] = 0
        Fy_rep[np.isnan(Fy_rep)] = 0
    return Fx_rep, Fy_rep

# Grid setup
x = np.linspace(0, 10, 50)
y = np.linspace(0, 10, 50)
X, Y = np.meshgrid(x, y)
goal = np.array([8, 8])
start = np.array([2, 2])
t = 1.0  # Time snapshot

# Moving obstacle (moves sinusoidally along x)
moving_obstacle = np.array([5 + np.sin(t), 5])
non_circular_obstacle = np.array([3, 7])

# Compute fields
Fx_att, Fy_att = compute_attractive_field(X, Y, goal)
Fx_rep1, Fy_rep1 = compute_repulsive_field(X, Y, moving_obstacle, kr=8)
Fx_rep2, Fy_rep2 = compute_repulsive_field(X, Y, non_circular_obstacle, kr=15, shape='ellipse')

# Combine all fields
Fx_total = Fx_att + Fx_rep1 + Fx_rep2
Fy_total = Fy_att + Fy_rep1 + Fy_rep2

# Plot vector field
plt.figure(figsize=(10, 10))
plt.streamplot(X, Y, Fx_total, Fy_total, color='blue', density=1.2)
plt.plot(*goal, 'go', label='Goal')
plt.plot(*start, 'ko', label='Start')

# Obstacles
circle1 = plt.Circle(moving_obstacle, 1, color='orange', alpha=0.6, label='Moving Obstacle')
ellipse = plt.matplotlib.patches.Ellipse(non_circular_obstacle, 3, 1.5, color='red', alpha=0.6, label='Elliptical Obstacle')
plt.gca().add_patch(circle1)
plt.gca().add_patch(ellipse)

plt.title("Advanced Potential Field with Moving & Non-Circular Obstacles")
plt.xlim(0, 10)
plt.ylim(0, 10)
plt.grid(True)
plt.axis('equal')
plt.legend()
plt.show()

# --- Placeholder for Local Minima Escape ---
# Strategies like random walk, simulated annealing, or wavefront expansion
# can be implemented here in the path planning logic

print("Next step: Implement local minima escape (wavefront/random walk) for smarter path planning.")